In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
#!unzip preprocessed_images.zip -d ./preprocessed_images

In [3]:
import os
import pandas as pd
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image

#transform = transforms.Compose([
#    transforms.Resize((224, 224)),
#    transforms.RandomHorizontalFlip(),
#    transforms.RandomRotation(20),
#    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
#    transforms.ToTensor(),
#    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#])

#full_df = pd.read_csv("full_df.csv")

#print(full_df.head())
#full_df.columns


In [4]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, Subset, WeightedRandomSampler
import numpy as np

class FundusDataset(Dataset):
    '''
    Custom Dataset para la carga de las imágenes de fondo de ojo con sus respectivos targets.
    Targets y sus significados:
    [1, 0, 0, 0, 0, 0, 0, 0] - Normal
    [0, 1, 0, 0, 0, 0, 0, 0] - Diabetes
    [0, 0, 1, 0, 0, 0, 0, 0] - Glaucoma
    [0, 0, 0, 1, 0, 0, 0, 0] - Cataratas
    [0, 0, 0, 0, 1, 0, 0, 0] - Degeneración macular asociada a la edad
    [0, 0, 0, 0, 0, 1, 0, 0] - Hipertensión
    [0, 0, 0, 0, 0, 0, 1, 0] - Miopía
    [0, 0, 0, 0, 0, 0, 0, 1] - Otras enfermedades o anormalidades
    '''
    def __init__(self, images_path, targets_df, transform=None):
        self.image_folder = ImageFolder(root=images_path, transform=transform)
        self.targets_df = targets_df

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        img, _ = self.image_folder[idx]
        img_filename = os.path.basename(self.image_folder.imgs[idx][0])
        target = np.array(self._map_label_to_target(self.targets_df.loc[self.targets_df['filename'] == img_filename, 'labels'].values[0]))
        return img, target

    def _map_label_to_target(self, label):
        if label == '[\'N\']':
            return [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        elif label == '[\'D\']':
            return [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        elif label == '[\'G\']':
            return [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        elif label == '[\'C\']':
            return [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
        elif label == '[\'A\']':
            return [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
        elif label == '[\'H\']':
            return [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
        elif label == '[\'M\']':
            return [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
        elif label == '[\'O\']':
            return [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
        else:
            raise Exception('Target desconocido')

full_df = pd.read_csv("full_df.csv")
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
dataset = FundusDataset(
    images_path='./preprocessed_images',
    targets_df=full_df,
    transform=data_transforms)

# COMENTAR ESTAS LÍNEAS: Reducir el tamaño total del dataset a los efectos
# de realizar pruebas rápidas.
# dataset = Subset(dataset, range(300))

# Dividir el dataset en entrenamiento y testing.
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Subsampling
labels_counts = full_df['labels'].value_counts()
sample_weights = [
    1. / labels_counts['[\'N\']'],
    1. / labels_counts['[\'D\']'],
    1. / labels_counts['[\'G\']'],
    1. / labels_counts['[\'C\']'],
    1. / labels_counts['[\'A\']'],
    1. / labels_counts['[\'H\']'],
    1. / labels_counts['[\'M\']'],
    1. / labels_counts['[\'O\']']
]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(dataset), replacement=True)

# Instanciar dataloaders para ambos subsets.
train_dataloader = DataLoader(train_dataset, batch_size=16, sampler=sampler, shuffle=False, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

# Verificar shapes de los batches de entrenamiento.
train_features, train_labels = next(iter(train_dataloader))
print(f'Train feature batch shape: {train_features.size()}')
print(f'Train labels batch shape: {train_labels.size()}')

# Verificar shapes de los batches de testing.
test_features, test_labels = next(iter(test_dataloader))
print(f'Test feature batch shape: {test_features.size()}')
print(f'Test labels batch shape: {test_labels.size()}')

Train feature batch shape: torch.Size([16, 3, 224, 224])
Train labels batch shape: torch.Size([16, 8])
Test feature batch shape: torch.Size([16, 3, 224, 224])
Test labels batch shape: torch.Size([16, 8])


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

num_classes = 8

model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(model)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).float().sum()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    
    return epoch_loss, epoch_accuracy


In [7]:
def validate_model(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).float().sum()
            total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    
    return epoch_loss, epoch_accuracy


In [8]:

num_epochs = 10

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(model, train_dataloader, criterion, optimizer, device)
    val_loss, val_accuracy = validate_model(model, test_dataloader, criterion, device)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

torch.save(model.state_dict(), 'vgg16_odir.pth')


Epoch 1/10
Train Loss: 0.1470, Train Accuracy: 7.7040
Val Loss: 4.4268, Val Accuracy: 6.7037
Epoch 2/10
Train Loss: 0.2355, Train Accuracy: 7.1519
Val Loss: 0.5209, Val Accuracy: 6.2127
Epoch 3/10
Train Loss: 0.2218, Train Accuracy: 7.0512
Val Loss: 0.5108, Val Accuracy: 7.0000
Epoch 4/10
Train Loss: 0.2222, Train Accuracy: 7.0299
Val Loss: 0.5340, Val Accuracy: 7.0000
Epoch 5/10
Train Loss: 0.2239, Train Accuracy: 7.0316
Val Loss: 0.5328, Val Accuracy: 7.0000
Epoch 6/10
Train Loss: 0.2242, Train Accuracy: 7.0216
Val Loss: 0.5419, Val Accuracy: 7.0000
Epoch 7/10
Train Loss: 0.2224, Train Accuracy: 7.0297
Val Loss: 0.5193, Val Accuracy: 7.0000
Epoch 8/10
Train Loss: 0.2195, Train Accuracy: 7.0493
Val Loss: 0.5182, Val Accuracy: 7.0000
Epoch 9/10
Train Loss: 0.2217, Train Accuracy: 7.0338
Val Loss: 0.5626, Val Accuracy: 6.2127
Epoch 10/10
Train Loss: 0.2207, Train Accuracy: 7.0277
Val Loss: 0.5492, Val Accuracy: 7.0000
